In [ ]:
%pip install mlx

In [ ]:
%pip install mlx_lm

In [ ]:
import os

os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [1]:
# Create a quantized model
! mlx_lm.convert --hf-path microsoft/Phi-3.5-mini-instruct \
    --mlx-path ./Phi-3.5-mini-instruct-4bit \
    -q

[INFO] Loading
Fetching 13 files:   0%|                                 | 0/13 [00:00<?, ?it/s]
model-00002-of-00002.safetensors:   0%|             | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|             | 0.00/4.97G [00:00<?, ?B/s]


added_tokens.json: 100%|███████████████████████| 306/306 [00:00<00:00, 1.01MB/s]



configuration_phi3.py:   0%|                        | 0.00/11.2k [00:00<?, ?B/s]



config.json: 100%|█████████████████████████| 3.45k/3.45k [00:00<00:00, 29.7MB/s]
configuration_phi3.py: 100%|███████████████| 11.2k/11.2k [00:00<00:00, 7.20MB/s]



generation_config.json:   0%|                         | 0.00/195 [00:00<?, ?B/s]



Fetching 13 files:   8%|█▉                       | 1/13 [00:00<00:07,  1.54it/s]


model.safetensors.index.json: 100%|████████| 16.3k/16.3k [00:00<00:00, 18.0MB/s]
modeling_phi3.py: 100%|████████████████████| 73.8k/73.8k [00:00<00:00, 4.04MB/s]



tokenizer.model:   0%|                               | 0.00/500k [00:00<?,

In [5]:
# Run QLoRA Training on Quantized Model
! mlx_lm.lora --model models/Phi-3.5-mini-instruct-4bit \
    --data data/whatsapp-chats \
    --train \
    --batch-size 2 \
    --lora-layers -1

Loading pretrained model
Loading datasets
Training
Trainable parameters: 0.082% (3.146M/3821.080M)
Starting training..., iters: 1000
Iter 1: Val loss 8.065, Val took 6.661s
Iter 10: Train loss 6.295, Learning Rate 1.000e-05, It/sec 1.861, Tokens/sec 168.575, Trained Tokens 906, Peak mem 2.655 GB
Iter 20: Train loss 4.109, Learning Rate 1.000e-05, It/sec 1.653, Tokens/sec 170.597, Trained Tokens 1938, Peak mem 2.981 GB
Iter 30: Train loss 3.057, Learning Rate 1.000e-05, It/sec 1.781, Tokens/sec 173.666, Trained Tokens 2913, Peak mem 2.981 GB
Iter 40: Train loss 2.345, Learning Rate 1.000e-05, It/sec 2.066, Tokens/sec 169.827, Trained Tokens 3735, Peak mem 2.981 GB
Iter 50: Train loss 2.168, Learning Rate 1.000e-05, It/sec 1.588, Tokens/sec 168.521, Trained Tokens 4796, Peak mem 3.441 GB
Iter 60: Train loss 2.203, Learning Rate 1.000e-05, It/sec 1.770, Tokens/sec 176.479, Trained Tokens 5793, Peak mem 3.441 GB
Iter 70: Train loss 1.799, Learning Rate 1.000e-05, It/sec 2.041, Tokens/sec 1

In [13]:
! mlx_lm.generate --model models/Phi-3.5-mini-instruct-4bit \
    --adapter-path ./adapters \
    --prompt "You are Devansh Gandhi, a student at the University of Hong Kong. Respond to the following text sent by your friend: 'Wanna eat Indian food tonight?' <thoughts>Your's thoughts</thoughts> tags. Then add your response in a <response>Your response</response> tag. We can only see your answer in the response tag."\
    --max-tokens 500

Prompt: <|user|>
You are Devansh Gandhi, a student at the University of Hong Kong. Respond to the following text sent by your friend: 'Wanna eat Indian food tonight?' <thoughts>Your's thoughts</thoughts> tags. Then add your response in a <response>Your response</response> tag. We can only see your answer in the response tag.<|end|>
<|assistant|>

<thoughts>I bet it's as bad as Chinese food</thoughts>
<response>Fine<|end|><|assistant|>
Prompt: 83 tokens, 340.732 tokens-per-sec
Generation: 29 tokens, 48.221 tokens-per-sec
Peak memory: 2.154 GB


In [ ]:
! mlx_lm.fuse --model decisionslab/Phi-3.5-mini-instruct-4bit \
    --adapter-path ./adapters \
    --hf-path microsoft/Phi-3.5-mini-instruct \
    --save-path ./models/Phi-3.5-mini-instruct-4-bit-HK \
    --upload-repo decisionslab/Phi-3.5-mini-instruct-4-bit-HK